# Import Agg Performance Nationwide Data

In [1]:
import cx_Oracle
import pandas as pd
import numpy as np

In [2]:
conStr = 'NARUT_E01044451/v9VzR8xsuwp#tyTb@TDM-scan.dc2.true.th:1521/TDMDBPR'
conn = cx_Oracle.connect(conStr)
print('TDMDBPR : Connected')
cur = conn.cursor()

v_week = (2023021, )


try:
    # print("Database version:", conn.version)
    print(f'\nExecute query as Week {v_week[0]}...')

    sqlTxt = """
        SELECT TM_KEY_DAY, TM_KEY_WK, TM_KEY_MTH, TM_KEY_QTR, TM_KEY_YR, DAY_DESC, MONTH_DESC, QUARTER_DESC, COMP_CD, METRIC_CD, METRIC_NAME, ACTUAL_AS_OF, AGG_TYPE, RR_IND, GRY_IND, UOM, PERIOD, ACTUAL_SNAP, TARGET_SNAP, BASELINE_SNAP, ACH_SNAP, GAP_SNAP, ACTUAL_AGG, TARGET_AGG, BASELINE_AGG, ACH_AGG, GAP_AGG, WOW, WOW_PERCENT, MOM, MOM_PERCENT, QOQ, QOQ_PERCENT, YOY, YOY_PERCENT, RR, RR_ACH, WTD, MTD, QTD, YTD, CENTER, METRIC_GRP, PRODUCT_GRP, SEQ, PRIORITY, REMARK, AREA_TYPE, AREA_CD, AREA_NAME, REMARK_ACTUAL, REMARK_TARGET, PPN_TM, METRIC_NAME_GROUP
        FROM GEOSPCAPPO.AGG_PERFORMANCE_NEWCO NOLOCK
        WHERE AREA_TYPE = 'P'
    """
    
    cur.execute(sqlTxt)

    print(f'\nDone !!!')

    rows = cur.fetchall()
    
    df = pd.DataFrame.from_records(rows, columns=[x[0] for x in cur.description])


except cx_Oracle.DatabaseError as e:
    print(f'Error with Oracle : {e}')


finally:
    cur.close()
    conn.close()
    print('\nTDMDBPR : Disconnected')

TDMDBPR : Connected

Execute query as Week 2023021...

Done !!!

TDMDBPR : Disconnected


In [3]:
df.shape

(45568, 54)

In [5]:
df.columns

Index(['TM_KEY_DAY', 'TM_KEY_WK', 'TM_KEY_MTH', 'TM_KEY_QTR', 'TM_KEY_YR',
       'DAY_DESC', 'MONTH_DESC', 'QUARTER_DESC', 'COMP_CD', 'METRIC_CD',
       'METRIC_NAME', 'ACTUAL_AS_OF', 'AGG_TYPE', 'RR_IND', 'GRY_IND', 'UOM',
       'PERIOD', 'ACTUAL_SNAP', 'TARGET_SNAP', 'BASELINE_SNAP', 'ACH_SNAP',
       'GAP_SNAP', 'ACTUAL_AGG', 'TARGET_AGG', 'BASELINE_AGG', 'ACH_AGG',
       'GAP_AGG', 'WOW', 'WOW_PERCENT', 'MOM', 'MOM_PERCENT', 'QOQ',
       'QOQ_PERCENT', 'YOY', 'YOY_PERCENT', 'RR', 'RR_ACH', 'WTD', 'MTD',
       'QTD', 'YTD', 'CENTER', 'METRIC_GRP', 'PRODUCT_GRP', 'SEQ', 'PRIORITY',
       'REMARK', 'AREA_TYPE', 'AREA_CD', 'AREA_NAME', 'REMARK_ACTUAL',
       'REMARK_TARGET', 'PPN_TM', 'METRIC_NAME_GROUP'],
      dtype='object')

In [6]:
df.dtypes

TM_KEY_DAY                    int64
TM_KEY_WK                     int64
TM_KEY_MTH                    int64
TM_KEY_QTR                    int64
TM_KEY_YR                     int64
DAY_DESC                     object
MONTH_DESC                   object
QUARTER_DESC                 object
COMP_CD                      object
METRIC_CD                    object
METRIC_NAME                  object
ACTUAL_AS_OF                float64
AGG_TYPE                     object
RR_IND                        int64
GRY_IND                      object
UOM                          object
PERIOD                       object
ACTUAL_SNAP                 float64
TARGET_SNAP                 float64
BASELINE_SNAP               float64
ACH_SNAP                    float64
GAP_SNAP                    float64
ACTUAL_AGG                  float64
TARGET_AGG                  float64
BASELINE_AGG                float64
ACH_AGG                     float64
GAP_AGG                     float64
WOW                         

In [7]:
df.describe()

,TM_KEY_DAY,TM_KEY_WK,TM_KEY_MTH,TM_KEY_QTR,TM_KEY_YR,ACTUAL_AS_OF,RR_IND,ACTUAL_SNAP,TARGET_SNAP,BASELINE_SNAP,...,BASELINE_AGG,ACH_AGG,GAP_AGG,WOW,WOW_PERCENT,MOM,MOM_PERCENT,RR,RR_ACH,SEQ
count,4.556800e+04,4.556800e+04,45568.000000,45568.000000,45568.0,1.778300e+04,45568.000000,1.651400e+04,8.051000e+03,6100.000000,...,2.613000e+04,9892.000000,9.892000e+03,1.586000e+04,15860.000000,1.250400e+04,12504.000000,2.661100e+04,4361.000000,41919.000000
mean,2.023041e+07,2.023015e+06,202303.966292,20231.651685,2023.0,2.023041e+07,0.277344,1.715522e+07,2.610342e+07,1031.032579,...,3.765279e+03,88.581171,-3.688487e+06,3.602490e+06,1.528768,1.849299e+05,-3.234970,3.209454e+08,117.385399,2743.456476
std,8.119929e+01,3.672565e+00,0.813511,0.476442,0.0,8.101813e+01,0.447693,6.818419e+07,9.182622e+07,7013.184675,...,6.258295e+04,96.303223,2.586139e+07,7.581670e+07,396.228980,1.489258e+07,407.410977,1.268820e+09,99.328296,1241.274154
min,2.023030e+07,2.023009e+06,202303.000000,20231.000000,2023.0,2.023030e+07,0.000000,-1.344830e+05,-2.254229e+02,0.000000,...,0.000000e+00,-378.260870,-3.145081e+08,-1.251551e+09,-33353.333333,-1.515675e+08,-14187.500000,-4.291976e+05,-53.465488,1001.000000
25%,2.023032e+07,2.023012e+06,202303.000000,20231.000000,2023.0,2.023032e+07,0.000000,3.444827e+01,2.250000e+00,0.000000,...,0.000000e+00,64.103274,-4.267118e+04,-1.348000e+03,-2.676062,-2.819750e+03,-4.288283,0.000000e+00,89.273475,2121.000000
50%,2.023041e+07,2.023015e+06,202304.000000,20232.000000,2023.0,2.023041e+07,0.000000,2.195000e+03,3.253097e+02,1.535000,...,0.000000e+00,92.424737,-2.760000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000e+00,103.624498,2263.000000
75%,2.023051e+07,2.023018e+06,202305.000000,20232.000000,2023.0,2.023050e+07,1.000000,3.628314e+06,1.763989e+06,70.000000,...,0.000000e+00,101.395953,9.000000e-02,2.299000e+03,2.812849,4.079500e+03,5.517840,1.660376e+05,125.185869,2467.000000
max,2.023053e+07,2.023021e+06,202305.000000,20232.000000,2023.0,2.023053e+07,1.000000,1.237689e+09,1.273999e+09,95485.000000,...,2.960035e+06,3687.508577,1.975063e+08,1.524454e+09,10122.535060,1.563949e+08,29907.142857,2.144701e+10,3687.508577,7028.000000


In [8]:
df.head()

,TM_KEY_DAY,TM_KEY_WK,TM_KEY_MTH,TM_KEY_QTR,TM_KEY_YR,DAY_DESC,MONTH_DESC,QUARTER_DESC,COMP_CD,METRIC_CD,...,SEQ,PRIORITY,REMARK,AREA_TYPE,AREA_CD,AREA_NAME,REMARK_ACTUAL,REMARK_TARGET,PPN_TM,METRIC_NAME_GROUP
0,20230301,2023009,202303,20231,2023,Wednesday,March,Quarter 1,TRUE,DIS10015,...,7015.0,High,None,P,P,Nationwide,None,None,2023-05-29 08:50:50,% After-Sales Service via Digital Channels
1,20230302,2023009,202303,20231,2023,Thursday,March,Quarter 1,TRUE,DIS10015,...,7015.0,High,None,P,P,Nationwide,None,None,2023-05-29 08:50:50,% After-Sales Service via Digital Channels
2,20230303,2023009,202303,20231,2023,Friday,March,Quarter 1,TRUE,DIS10015,...,7015.0,High,None,P,P,Nationwide,None,None,2023-05-29 08:50:50,% After-Sales Service via Digital Channels
3,20230304,2023009,202303,20231,2023,Saturday,March,Quarter 1,TRUE,DIS10015,...,7015.0,High,None,P,P,Nationwide,None,None,2023-05-29 08:50:50,% After-Sales Service via Digital Channels
4,20230305,2023009,202303,20231,2023,Sunday,March,Quarter 1,TRUE,DIS10015,...,7015.0,High,None,P,P,Nationwide,None,None,2023-05-29 08:50:50,% After-Sales Service via Digital Channels


## Generate Output file

In [4]:
# C:\Users\Narut4\coding\Jupyter\Work with Oracle.ipynb

output_name = f'Raw-Agg Perf Nationwide_Wk {v_week[0]}'

df.to_csv(f'{output_name}.csv', index=False, encoding='utf-8')

print(f'\n"{output_name}.csv" is generated')


"Raw-Agg Perf Nationwide_Wk 2023021.csv" is generated
